In [2]:
import hopsworks
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

In [3]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
api = 'UtYWT9JBE4jbsOVW.dzfTExU7QMCzzR51EADTOZCXBzl0VmgB2y012yd8nFTG6v1VHgWazdx2a2SuJAY1'
project = hopsworks.login(api_key_value = api)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/229528
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# wine_fg = fs.get_feature_group(name="wine_2", version=16)
traffic_fg = fs.get_or_create_feature_group(
    name="sf_trarric_accidents",
    version=1,
    primary_key=['weather_1_mapped', 'intersection_mapped',
    'road_cond_1_mapped', 'control_device_mapped',
    'lighting_mapped', 'road_surface_mapped',
    'type_of_collision_mapped', 'time_cat_mapped',
    'party2_move_pre_acc_mapped', 'party1_move_pre_acc_mapped', 'ped_action_mapped',
    'party1_type_mapped', 'party2_type_mapped',
    'mviw_mapped','collision_severity_class'], 
    description="Incident dataset")
query = traffic_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="data_1",
                                  version=1,
                                  description="Read from SF traffic dataset",
                                  labels=["collision_severity_class"],
                                  query=query)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/229528/fs/229447/fv/data_1/version/1


In [5]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)
print("length of x y_train max:",y_train.max())
print("length of x X_train:",X_train.shape)
print("length of x X_test:",X_test.shape)
from sklearn.linear_model import LogisticRegression
# Train our model with the Scikit-learn K-nearest-neighbors algorithm using our features (X_train) and labels (y_train)
from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(random_state = 0, n_estimators = 150,
#                                     criterion = 'entropy')
model = LogisticRegression(random_state = 0, penalty = 'l2',max_iter=10000 )
model.fit(X_train, y_train.values.ravel())

Finished: Reading data from Hopsworks, using Hive (5.23s) 


length of x y_train max: collision_severity_class    2
dtype: int64
length of x X_train: (19687, 14)
length of x X_test: (4922, 14)


LogisticRegression(max_iter=10000, random_state=0)

In [6]:
X_train

,weather_1_mapped,intersection_mapped,road_cond_1_mapped,control_device_mapped,lighting_mapped,road_surface_mapped,type_of_collision_mapped,time_cat_mapped,party2_move_pre_acc_mapped,party1_move_pre_acc_mapped,ped_action_mapped,party1_type_mapped,party2_type_mapped,mviw_mapped
0,2,1,1,0,1,2,0,0,0,1,0,2,0,5
2,0,0,1,0,1,0,5,3,4,5,0,2,5,2
4,0,2,6,1,1,0,3,0,11,1,0,0,1,6
5,0,0,1,0,1,0,3,0,3,1,0,0,1,6
7,4,1,1,0,0,2,0,1,0,0,0,2,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24604,2,0,1,1,1,2,3,3,3,1,0,2,1,4
24605,0,0,1,1,0,0,7,1,3,1,4,2,1,4
24606,0,0,1,0,0,2,1,1,8,5,0,2,3,1
24607,4,0,1,1,0,2,5,1,3,5,2,2,3,1


In [7]:
y_train.value_counts()

collision_severity_class
0                           11731
1                            6169
2                            1787
Name: count, dtype: int64

In [8]:
y_test.value_counts()

collision_severity_class
0                           2883
1                           1602
2                            437
Name: count, dtype: int64

In [9]:
y_pred = model.predict(X_test)
y_pred.min()

0

In [17]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state = 0, penalty = 'l2',max_iter=15000 )
model.fit(X_train, y_train)

# Predicting Test Set
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

results = pd.DataFrame([['Logistic Regression', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
print(results)

                 Model  Accuracy  Precision    Recall  F1 Score
0  Logistic Regression  0.593864   0.510433  0.593864  0.485928


In [20]:
from sklearn.neural_network import MLPClassifier

# Define the MLPClassifier
# Here, we are using a simple architecture with two hidden layers, each with 100 neurons
# You can experiment with different architectures and hyperparameters
model = MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=1000, random_state=42)

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
mlp_pred = model.predict(X_test)

# Evaluate the model
print("MLP Classification Report:\n", classification_report(y_test, mlp_pred))

# Predicting Test Set
# y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(y_test, mlp_pred)
prec = precision_score(y_test, mlp_pred, average='weighted')
rec = recall_score(y_test, mlp_pred, average='weighted')
f1 = f1_score(y_test, mlp_pred, average='weighted')

results = pd.DataFrame([['Logistic Regression', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
print(results)

MLP Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.74      0.68      2883
           1       0.40      0.31      0.35      1602
           2       0.12      0.07      0.09       437

    accuracy                           0.54      4922
   macro avg       0.38      0.38      0.37      4922
weighted avg       0.51      0.54      0.52      4922

                 Model  Accuracy  Precision    Recall  F1 Score
0  Logistic Regression  0.543478   0.510276  0.543478  0.522085


In [12]:
## Randomforest
from sklearn.ensemble import RandomForestClassifier
import numpy as np
model = RandomForestClassifier(random_state = 0, n_estimators = 150,
                                    criterion = 'entropy')
model.fit(X_train, y_train)

# Predicting Test Set
y_pred = model.predict(X_test)
print("y_pred max is:",y_train.max())
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

model_results = pd.DataFrame([['Random Forest (n=100)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])


print(model_results)

y_pred max is: collision_severity_class    2
dtype: int64
                   Model  Accuracy  Precision    Recall  F1 Score
0  Random Forest (n=100)  0.565421   0.517442  0.565421  0.531342


In [10]:
# from sklearn.ensemble import RandomForestClassifier
# classifier = RandomForestClassifier(random_state = 0, n_estimators = 150,
#                                     criterion = 'entropy')
# y_train_1d = y_train.values.ravel() if isinstance(y_train, pd.DataFrame) else y_train.ravel()
# y_test_1d = y_test.values.ravel() if isinstance(y_test, pd.DataFrame) else y_test.ravel()

# classifier.fit(X_train, y_train)

# # Predicting Test Set
# y_pred = classifier.predict(X_test)
# print("y_pred max is:",y_train.max())
# acc = accuracy_score(y_test, y_pred)
# prec = precision_score(y_test, y_pred,average='weighted')
# rec = recall_score(y_test, y_pred,average='weighted')
# f1 = f1_score(y_test, y_pred,average='weighted')

# model_results = pd.DataFrame([['Random Forest (n=100)', acc, prec, rec, f1]],
#                columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])


# print(model_results)

y_pred max is: quality    2
dtype: int64
                   Model  Accuracy  Precision    Recall  F1 Score
0  Random Forest (n=100)  0.730769   0.712958  0.730769  0.721716


In [13]:
# Evaluate model performance using the features from the test set (X_test)
y_pred = model.predict(X_test)

# Compare predictions (y_pred) with the labels in the test set (y_test)
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred)
print(acc)

0.5654205607476636


In [18]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
model_dir="sf_traffic_models"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/sf_traffic_model_1.pkl")
# fig.savefig(model_dir + "/confusion_matrix.png")    

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)
# Ensure that the metrics are single numerical values
sf_traffic_model = mr.python.create_model(
    name="sf_traffic_model_1", 
    metrics={
        "accuracy": model_results['Accuracy'].iloc[0],
        "Precision": model_results['Precision'].iloc[0],
        "Recall": model_results['Recall'].iloc[0],
        "F1_score": model_results['F1 Score'].iloc[0]
    },
    model_schema=model_schema,
    description="SF traffic model"
)

# Upload the model to the model registry, including all files in 'model_dir'
sf_traffic_model.save(model_dir)


Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 1887/1887 elapsed<00:01 remaining<00:00:01,  2.92it/s]
Uploading: 100.000%|██████████| 1315/1315 elapsed<00:01 remaining<00:000:04,  1.10s/it]
Model export complete: 100%|██████████| 6/6 [00:09<00:00,  1.57s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/229528/models/sf_traffic_model_1/1


Model(name: 'sf_traffic_model_1', version: 1)